In [1]:
%run functions.py

Using TensorFlow backend.


In [2]:
stemmer = StempelStemmer.polimorf()

Loading:  99%|█████████████████████████████████████████████████████▋| 11311188/11368252 [00:19<00:00, 607167.44bytes/s]

In [3]:
xall,yall = loadDataFromCSV('data.csv')
number_of_classes = 25

Loading: 100%|██████████████████████████████████████████████████████| 11368252/11368252 [00:20<00:00, 553535.14bytes/s]


In [4]:
xall,yall=deleteBasedOnIndicesList(xall,yall)

In [5]:
print("Set size: ", len(xall))

Set size:  29423


In [6]:
for i in range(0,len(yall)):
    yall[i] = f(float(yall[i]))
for i in range(0,len(xall)):
    xall[i] = str(xall[i])

In [7]:
number_of_samples_for_each_class = count_number_of_samples_for_each_class(yall)
weights = assign_weight_for_each_class(len(yall), number_of_samples_for_each_class)
class_weight = {0: weights[0],
                1: weights[1],
                2: weights[2],
                3: weights[3],
                4: weights[4],
                5: weights[5],
                6: weights[6],
                7: weights[7],
                8: weights[8],
                9: weights[9],
                10: weights[10],
                11: weights[11],
                12: weights[12],
                13: weights[13],
                14: weights[14],
                15: weights[15],
                16: weights[16],
                17: weights[17],
                18: weights[18],
                19: weights[19],
                20: weights[20],
                21: weights[21],
                22: weights[22],
                23: weights[23],
                24: weights[24],
               }

In [8]:
number_of_samples_for_each_class

[1625,
 434,
 132,
 847,
 141,
 861,
 1630,
 1183,
 178,
 1935,
 165,
 2000,
 2000,
 2000,
 320,
 2001,
 671,
 1240,
 2003,
 2002,
 18,
 2002,
 39,
 1996,
 2000]

In [9]:
yall=to_categorical(yall)

In [10]:
stop_words = set(stopwords.words('polish'))
processed_reviews = []
single_review = "string to iniialize <br /> my email id is charilie@waoow.com. You can also reach to me at charlie's"

for review in range(0,len(xall)):
    post_stemming = []
    single_review = xall[review]
    single_review = single_review.lower()
    single_review = re.sub('<.*?>',' ',single_review)
    single_review = re.sub('\W',' ',single_review)
    single_review = re.sub('\s+[a-zA-Z]\s+',' ', single_review)
    single_review = re.sub('\s+',' ', single_review)       
    word_tokens = word_tokenize(single_review)    
    filtered_sentence = " ".join([w for w in word_tokens if w not in stop_words])
    arr = Convert(filtered_sentence)
    if(arr[0] or len(arr)>1):
        for word in arr:
            post_stemming.append(stemmer.stem(word))
    single_review = listToString(post_stemming)    
    processed_reviews.append(filtered_sentence)    
print(processed_reviews[0])

porządku polecam


In [11]:
x_train, x_test, y_train, y_test = train_test_split(processed_reviews,yall,test_size=0.2,random_state=42)
print("Train-set size: ", len(x_train))
print("Test-set size:  ", len(x_test))

Train-set size:  23538
Test-set size:   5885


In [12]:
data_text = x_train + x_test
num_words = 1000
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(data_text)
tokenizer.word_index
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [13]:
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

36

In [14]:
pad = 'pre'
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens,
                            padding=pad, truncating=pad)

x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens,
                           padding=pad, truncating=pad)

In [15]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [16]:
embedding_size = 8

In [17]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(1000,64),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_size)),
    tf.keras.layers.Dense(embedding_size, activation='relu'),
    tf.keras.layers.Dense(number_of_classes, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          64000     
_________________________________________________________________
bidirectional (Bidirectional (None, 16)                4672      
_________________________________________________________________
dense (Dense)                (None, 8)                 136       
_________________________________________________________________
dense_1 (Dense)              (None, 25)                225       
Total params: 69,033
Trainable params: 69,033
Non-trainable params: 0
_________________________________________________________________


In [18]:
optimizer = Adam(lr=0.01)

In [19]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [20]:
%%time
y_train=np.array(y_train)
model.fit(x_train_pad, y_train, class_weight=class_weight, validation_split=0.05, epochs=8, batch_size=number_of_classes)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 22361 samples, validate on 1177 samples
Epoch 1/8
22361/22361 [==============================] - 33s 1ms/sample - loss: 2.9998 - acc: 0.0983 - val_loss: 3.2904 - val_acc: 0.1393
Epoch 2/8
22361/22361 [==============================] - 26s 1ms/sample - loss: 2.7783 - acc: 0.1285 - val_loss: 3.1913 - val_acc: 0.1198
Epoch 3/8
22361/22361 [==============================] - 27s 1ms/sample - loss: 2.6989 - acc: 0.1381 - val_loss: 3.1601 - val_acc: 0.1504
Epoch 4/8
22361/22361 [==============================] - 26s 1ms/sample - loss: 2.6344 - acc: 0.1389 - val_loss: 3.1103 - val_acc: 0.1003
Epoch 5/8
22361/22361 [==============================] - 28s 1ms/sample - loss: 2.5688 - acc: 0.1340 - val_loss: 3.1125 - val_acc: 0.1003
Epoch 6/8
22361/22361 [==============================] - 35s 2ms/sample - loss: 2.5179 - acc: 0.1356 - val_loss: 3.1959 - val_acc: 0.0943
Epoch 7/8
22361/22361 [==============================] - 33s 1ms/sample -

In [21]:
%%time
y_pred = model.predict(x=x_test_pad)

Wall time: 3.57 s


In [22]:
y_res=[]
y_test_res=[]
##pred
for i in range(0,len(y_pred)):
    rate=classifyRate(y_pred[i])
    y_res.append(rate)

for i in range(0,len(y_test)):
    rate=np.argmax(y_test[i])
    y_test_res.append(rate)

NameError: name 'classifyRate' is not defined

In [ ]:
def classifyRate(rate):
    classifiedRate = np.argmax(rate)
    rate2 = rate.copy()
    maks = max(rate)
    rate2 = rate2[rate2 != maks]
    maks2 = max(rate2)
    maksIndex = np.where(rate == maks)[0][0]
    maksIndex2 = np.where(rate == maks2)[0][0] 
    result = -1
    temp = findBestRate(rate, maksIndex)
    temp2 = findBestRate(rate, maksIndex2)
    if (temp>temp2):
        result = maksIndex
    else:
        result = maksIndex2
    #if (i>0 and i < len(rate)-1):
     #   temp = rate[i] + 0.5*rate[i-1] + 0.5*rate[i+1]
    #elif (i == 0):
    #    temp = rate[i] + 0.5*rate[i+1]
    #else:
    #    temp = rate[i] + 0.5*rate [i-1]
    #if (temp>maks):
    #    maks = temp
    return result

In [ ]:
def findBestRate(rate, maksIndex):
    if (maksIndex>0 and maksIndex<len(rate)-1):
        temp = rate[maksIndex]+0.5*rate[maksIndex-1]+0.5*rate[maksIndex+1]
    elif (maksIndex == 0):
        temp = rate[maksIndex] + 0.5*rate[maksIndex+1]
    else:
        temp = rate[maksIndex] + 0.5*rate[maksIndex-1]
    return temp

In [ ]:
def classifyRate(rate):
    classifiedRate = np.argmax(rate)
    maks = 0
    maks2 = 0
    temp = 0
    maxIndex = 0
    for i in range (0, len(rate)):        
        if (i>0 and i < len(rate)-1):
            temp = rate[i] + 0.5*rate[i-1] + 0.5*rate[i+1]
        elif (i == 0):
            temp = rate[i] + 0.5*rate[i+1]
        else:
            temp = rate[i] + 0.5*rate [i-1]
        if (temp>maks):
            maks = temp
            maxIndex = i
    return maxIndex

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
cm=confusion_matrix(y_test_res,y_res)
cm_display = ConfusionMatrixDisplay(cm).plot()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
cm=confusion_matrix(y_test_res,y_res)
cm_display = ConfusionMatrixDisplay(cm).plot()
fig, ax = plt.subplots(figsize=(15,15))
cm_display.plot(ax=ax)

In [ ]:
listOfTP = [0] * 25
listOfFP = [0] * 25
listOfTN = [0] * 25
listOfFN = [0] * 25

In [ ]:
for i in range(0,len(y_test_res)):
    y_test_res[i] = g(float(y_test_res[i]))
for i in range(0,len(y_res)):
    y_res[i] = g(float(y_res[i]))

In [ ]:
for i in range(len(listOfTP)): 
    listOfTP[i], listOfFP[i], listOfTN[i], listOfFN[i] = perf_measure(y_test_res, y_res, y_test_res[i])

In [ ]:
sensitivity = [0] * 25
specificity = [0] * 25
precision = [0] * 25
NPV = [0] * 25

In [ ]:
for i in range(len(sensitivity)): 
    sensitivity[i], specificity[i], precision[i], NPV[i] = calculate_metrics(listOfTP[i], listOfFP[i], listOfTN[i], listOfFN[i])

In [ ]:
sensitivity

In [ ]:
specificity

In [ ]:
precision

In [ ]:
NPV

In [ ]:
myset = set(y_test_res)
print(myset)

In [ ]:
mylist = list(myset)
print (mylist)

In [ ]:
listOfTP